<a href="https://colab.research.google.com/github/bitratek/CNN_Stuff/blob/main/Optimized_CNNModel_KerasTuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
import tensorflow as tf
from tensorflow import keras
import numpy as np


In [25]:
print(tf.__version__)

2.9.2


In [26]:
fashion_mnist = keras.datasets.fashion_mnist

In [27]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [28]:
print(test_images[0].shape)

(28, 28)


In [29]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [30]:
def build_model(hp):
  model = keras.Sequential([
      keras.layers.Conv2D(
          filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
          kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
          activation='relu',
          input_shape=(28,28,1)
      ),
      keras.layers.Conv2D(
          filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
          kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
          activation='relu',
      ),
      keras.layers.Flatten(),
      keras.layers.Dense(
          units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
          activation='relu'
      ),
      keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values = [1e-2, 1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy']
  )

  return model

In [31]:
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

In [32]:
tuner_search = RandomSearch(build_model, objective='val_accuracy', max_trials=5, directory='output', project_name="Mnist Fashion")

tuner_search.search(train_images, train_labels, epochs=3, validation_split=0.1)

In [33]:
model = tuner_search.get_best_models(num_models=1)[0]

In [34]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 80)        2080      
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 32)        23072     
                                                                 
 flatten (Flatten)           (None, 15488)             0         
                                                                 
 dense (Dense)               (None, 80)                1239120   
                                                                 
 dense_1 (Dense)             (None, 10)                810       
                                                                 
Total params: 1,265,082
Trainable params: 1,265,082
Non-trainable params: 0
_________________________________________________________________


In [35]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2307 - accuracy: 0.9125 - val_loss: 0.3349 - val_accuracy: 0.8855
Epoch 5/10
1688/1688 [==============================] - 9s 6ms/step - loss: 0.1955 - accuracy: 0.9259 - val_loss: 0.3796 - val_accuracy: 0.8858
Epoch 6/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1660 - accuracy: 0.9377 - val_loss: 0.4298 - val_accuracy: 0.8753
Epoch 7/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1403 - accuracy: 0.9474 - val_loss: 0.4751 - val_accuracy: 0.8782
Epoch 8/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1223 - accuracy: 0.9539 - val_loss: 0.5476 - val_accuracy: 0.8875
Epoch 9/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1063 - accuracy: 0.9619 - val_loss: 0.5781 - val_accuracy: 0.8868
Epoch 10/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0967 - accuracy: 0.9659 - val_loss: 0.6113 - val_accuracy